In [ ]:
!pip install numdifftools
import numdifftools as ndt

     |████████████████████████████████| 962kB 8.0MB/s 


In [ ]:
import numpy as np
import pandas as pd
import statistics as st
import math
#import PIL as pil
from zipfile import ZipFile as zipf
from PIL import Image as img

In [ ]:
#Mount drive
from google.colab import drive
drive.mount('/content/drive')
drive_path = "/content/drive/MyDrive/Assignment1_ELL409/Q3/Medical_MNIST.zip"

Mounted at /content/drive


In [ ]:
def confusion_matrx(out_true_dict,num_class):
  conf_m = np.zeros(dtype=int,shape=(num_class,num_class))
  
  for i in out_true_dict.keys():
    true_v = i
    pred_vs = out_true_dict[i]
    for pred_v in pred_vs:
      conf_m[true_v][pred_v] +=1

  acc = 0
  for i in range(num_class):
    acc += conf_m[i]
  acc = acc/sum(sum(conf_m))
  return conf_m

In [ ]:
categories = {"AbdomenCT":0,"BreastMRI":1,"CXR":2,"ChestCT":3,"Hand":4,"HeadCT":5}
all_vals = {0:[],1:[],2:[],3:[],4:[],5:[]}

with zipf(drive_path) as archive:
  for entry in archive.infolist():
    with archive.open(entry) as file:
      key = file.name.split('/')[0]
      imag = img.open(file)
      #imag = imag.resize((32,32))
      #print(imag.size, imag.mode, len(imag.getdata()))
      #print()
      all_vals[categories[key]].append(np.asarray(imag.getdata()))

In [ ]:
lst = all_vals[0]
k = int(len(lst)*0.2)
train_set = lst[k:]
#train_set.extend(lst[5*k:])
test_set = lst[:k]
del(lst)

end_indxs = [(len(test_set)-1,len(train_set)-1)]

for key in range(1,6):
  lst = all_vals[key]
  k = int(len(lst)*0.2)
  
  train_set.extend(lst[k:])
  #train_set.extend(lst[5*k:])

  test_set.extend(lst[:k])
  end_indxs.append((len(test_set)-1,len(train_set)-1))
  del(lst)

print(end_indxs)


[(1999, 7999), (3789, 15163), (5789, 23163), (7789, 31163), (9789, 39163), (11789, 47163)]


In [ ]:
del(all_vals)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(train_set)
train_set = scaler.transform(train_set)
test_set = scaler.transform(test_set)

In [ ]:
from sklearn.decomposition import PCA
# Make an instance of the Model
pca = PCA(n_components=3)
pca.fit(train_set)

PCA(copy=True, iterated_power='auto', n_components=3, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [ ]:
train_set = pca.transform(train_set)
test_set = pca.transform(test_set)

In [ ]:
traindict = {0:[],1:[],2:[],3:[],4:[],5:[]}
testdict = {0:[],1:[],2:[],3:[],4:[],5:[]}
testdict[0] = test_set[:end_indxs[0][0]+1]
traindict[0] = train_set[:end_indxs[0][1]+1]

for i in range(1,len(end_indxs)):
  testdict[i] = test_set[end_indxs[i-1][0]+1:end_indxs[i][0]+1]
  traindict[i] = train_set[end_indxs[i-1][1]+1:end_indxs[i][1]+1]


In [ ]:
d = {"train":traindict,"test":testdict}
import pickle
allv_dir = "/content/drive/MyDrive/Assignment1_ELL409/Q3/PCA_0test.pickle"
with open(allv_dir,"wb") as f:
  pickle.dump(d,f)

In [ ]:
import pickle
allv_dir = "/content/drive/MyDrive/Assignment1_ELL409/Q3/PCA_1test.pickle"
with open(allv_dir,"rb") as f:
  datas = pickle.load(f)


In [ ]:
import pickle
'''
allv_dir = "/content/drive/MyDrive/Assignment1_ELL409/Q3/all_medicalM_dict.pickle"
with open(allv_dir,"wb") as f:
  pickle.dump(all_vals,f)

data_splits = {0:[],1:[],2:[],3:[],4:[],5:[]}

for key in all_vals.keys():
  data = all_vals[key]
  r = 0.2
  len_1s = int(len(data)*r)
  lst_tr = []
  for i in range(5):
    start = i*len_1s
    end = (i+1)*len_1s
    if(i==4):
      end = len(data)

    lst_tr.append(data[start:end])
  data_splits[key] = lst_tr

with open("/content/drive/MyDrive/Assignment1_ELL409/Q3/32_split_medicalM_dict.pickle","wb") as f:
  pickle.dump(data_splits,f)
'''

In [ ]:
'''
allv_dir = "/content/drive/MyDrive/Assignment1_ELL409/Q3/split_medicalM_dict.pickle"

with open(allv_dir,"rb") as f:
  data_splits = pickle.load(f)
'''

In [ ]:
'''
def standardize_data(train_d,test_d,eps=10^-3):
  mu_arr = np.mean(train_d,axis=0)
  sd_arr = np.std(train_d,axis=0)
  for val in range(len(sd_arr)):
    if(sd_arr[val]==0):
      sd_arr[val] = eps
  
  for val in range(len(train_d)):
    lst_tr = train_d[val]
    
    lst_tr = np.subtract(lst_tr,mu_arr)
    train_d[val] = np.divide(lst_tr,sd_arr)
    
  for val in range(len(test_d)):
    lst_ts = test_d[val]
    lst_ts = np.subtract(lst_ts,mu_arr)
    test_d[val] = np.divide(lst_ts,sd_arr)

  scaler = StandardScaler()
  # Fit on training set only.
  scaler.fit(train_img)
  # Apply transform to both the training set and the test set.
  train_img = scaler.transform(train_img)
  test_img = scaler.transform(test_img)

  return train_d,test_d

def std_dicts(train_dict,test_dict):
  for key in train_dict.keys():
    train_dict[key],test_dict[key] = standardize_data(train_dict[key],test_dict[key])
  return train_dict,test_dict
'''

In [ ]:
train_dict = {}
test_dict = {}

for k in data_splits.keys():
  lst = data_splits[k]
  test_dict[k] = lst[0]
  val = lst[1]
  for i in range(2,5):
    val.append(lst[i])
  train_dict[k] = val


In [ ]:
train_dict,test_dict = std_dicts(train_dict,test_dict)

In [ ]:
import random

def split_data(data, r,flag): 
    '''r: ratio of split, r indicates amount in train; flag indicates if random or linear split'''
    train_len = int(len(data) * r) 
    test_len = len(data) - train_len
    test = [] 
    train = []

    if(flag==0):
      indxs = random.sample(range(0, len(data)), test_len)
      for v in range(len(data)):
        if v in indxs:
          test.append(data[v])
        else:
          train.append(data[v])

    else:
      train = data
      while len(test) < test_len:
        test.append(train.pop(0))

    return train, test 

In [ ]:
traindict = {0:[],1:[],2:[],3:[],4:[],5:[]}
testdict = {0:[],1:[],2:[],3:[],4:[],5:[]}

for k in all_vals.keys():
  lst = all_vals[k]
  print(len(lst))
  train,test = split_data(lst,0.8,0)
  traindict[k] = np.asarray(train)
  #testdict[k] = np.asarray(train[int(len(train)/2):])
  testdict[k] = np.asarray(test)
#print(traindict)
#print(testdict)

In [ ]:
traindict = datas["train"]
testdict = datas["test"]

In [ ]:
def MLE_Gaussian_Params(data_lst):
  n = data_lst.shape[0] #number of data points
  k = data_lst.shape[1] #feature vector dimension
  mu = data_lst[0][:]
  for j in range(1,n):
    mu = mu + data_lst[j][:]
  mu = mu/n
  
  sd = np.zeros(shape=(k,k),dtype=float)
  #print(k)
  for j in range(n):
    tmp = data_lst[j][:] - mu
    #print(np.transpose(tmp[np.newaxis]).shape)
    sd += np.matmul(np.transpose(tmp[np.newaxis]),tmp[np.newaxis])
  
  sd = sd/n
  
  return mu,sd

#@jit
def PDF_normaldistr(inpt,mu,sd):
  #mu = lst[0]
  #sd = lst[1]
  #print("in pdf")
  tmp = inpt-mu
  print(tmp)
  tmp = tmp[np.newaxis]
  k = tmp.shape[1]
  dt = np.linalg.det(sd)
  if(dt!=0):
    #print("det nonzero")    
    sdi = np.linalg.inv(sd)
    expt = np.matmul(tmp,sdi)
    expt = np.matmul(expt,np.transpose(tmp))*(-0.5)
    expt = math.exp(expt[0][0])
   # print("---")
   # print(expt)
   # print(((2*math.pi)**(-0.5*k))*((dt)**(-0.5)))
   # print("---")
    return expt*((2*math.pi)**(-0.5*k))*((dt)**(-0.5))
  
  else:
    print("0 det")
    sd1 = sd + np.random.rand(sd.shape[0],sd.shape[1])*5
    sdi = np.linalg.inv(sd1)
    dt = np.linalg.det(sd1)
    print(sdi)
    print(dt)
    expt = np.matmul(tmp,sdi)
    expt = np.matmul(expt,np.transpose(tmp))*(-0.5)
    print(expt)
    expt = math.exp(expt[0])
    return expt*((2*math.pi)**(-0.5*k))*((dt)**(-0.5))

  #print(expt)

def predict_label_MLEGauss(lst,lst_params,prior_arr):
  out_lab = (-1,0)

  for indx in range(len(lst_params)):
    theta = lst_params[indx]
    fx_y = PDF_normaldistr(lst,theta[0],theta[1])
    fx_y = prior_arr[indx]*fx_y
    if(out_lab[0]==-1):
      out_lab = (fx_y,indx)
    else:
      if(out_lab[0]<fx_y):
        out_lab = (fx_y,indx)

  return out_lab[1]

def Classify_MLE_Gauss(train_data_dict,test_data_dict,prior_arr):
  lst_params = []
  #true_class = []
  for k in train_data_dict.keys():
    lst_d = train_data_dict[k]
    mu,sd = MLE_Gaussian_Params(lst_d)
    lst_params.append((mu,sd))
    #true_class.append(k)
  
  #print(lst_params)
  
  dict_true_out = {}
  for ky in test_data_dict.keys():
    lst_d = test_data_dict[ky]
    out_list = []
    for arr in lst_d:
      out_l = predict_label_MLEGauss(arr,lst_params,prior_arr)
      out_list.append(out_l)
    
    dict_true_out[ky] = out_list
    
  return dict_true_out


In [ ]:
mu = np.zeros(shape=(3,))
sd = np.ones(shape=(3,3))
PDF_normaldistr(np.asarray([0.5,0.5,1]),mu,sd)

In [ ]:
prior_arr = [1/6,1/6,1/6,1/6,1/6,1/6]
output = Classify_MLE_Gauss(traindict,testdict,prior_arr)
print(output)

In [ ]:
print(confusion_matrx(output,6))

[[2000    0    0    0    0    0]
 [   0 1790    0    0    0    0]
 [   0    0 1976    0   23    1]
 [   0    0    0 2000    0    0]
 [   2    1   60    0 1886   51]
 [   0  113    0    0   54 1833]]


Bayes with multivariate normal, data downsampled NOT PCA

EM for GMMs,  Bayes 

Naive Bayes Experiments

Expts with MAP

Expts with Parzen